# 1. Prepare the tagged data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
from pathlib import Path

In [ ]:
# Read all the text names in one folder
dir = '/content/drive/MyDrive/GR/Tagged Amazon product review'
dataset_txt_paths = sorted([path for path in Path(dir).iterdir() if path.suffix in {'.txt'}])

# Check the number of files in the dataset
print('the dataset contains {} files'.format(len(dataset_txt_paths)))

the dataset contains 17 files


In [ ]:
# Check the names
for data_path in dataset_txt_paths:
    print(data_path)

/content/drive/MyDrive/GR/Tagged Amazon product review/Apex AD2600 Progressive-scan DVD player.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Canon G3.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Canon PowerShot SD500.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Canon S100.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Computer.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Creative Labs Nomad Jukebox Zen Xtra 40GB.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Diaper Champ.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Hitachi router.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Linksys Router.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/MicroMP3.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Nikon coolpix 4300.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Nokia 6600.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Nokia 6610.txt
/conten

In [ ]:
# Some raw data is not clean. Use this to check if one line contains multiple or no "##" (Done)
# If it does not output sentences, then no problem
# But this code cannot be skipped
read = []

for txt_path in dataset_txt_paths:
    with open(txt_path) as f:
        print(txt_path)
        count = 0
        for eachline in f:
            #print(eachline)
            read.append(eachline)
            count += 1
            if eachline.count('##') > 1:
                print(eachline)
                print(count)
            elif eachline.count('##') == 0 and eachline[:3] != '[t]':
                print(eachline)
                print(count)

            #if eachline.count('##') != 0:
            #    start_index = eachline.index('##')
            #    senti = eachline[:start_index]
            #    pos = '+' in senti
            #    neg = '-' in senti
            #    if not pos and not neg and len(senti)>1:
            #        #print(senti)
            #        print(eachline)
            #        print(count)

/content/drive/MyDrive/GR/Tagged Amazon product review/Apex AD2600 Progressive-scan DVD player.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Canon G3.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Canon PowerShot SD500.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Canon S100.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Computer.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Creative Labs Nomad Jukebox Zen Xtra 40GB.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Diaper Champ.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Hitachi router.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Linksys Router.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/MicroMP3.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Nikon coolpix 4300.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Nokia 6600.txt
/content/drive/MyDrive/GR/Tagged Amazon product review/Nokia 6610.txt
/conten

In [ ]:
# Here, we concatenate sentences in the same review (devided by [t])
all_reviews = []
t_index = []

for i in range(len(read)):
    if read[i][:3] == '[t]':
        t_index.append(i)

for i in range(len(t_index) - 1):
    all_reviews.append(read[t_index[i]: t_index[i + 1]])
all_reviews.append(read[t_index[-1]:])

all_reviews

[['[t] troubleshooting ad-2500 and ad-2600 no picture scrolling b/w . \n',
  '##repost from january 13 , 2004 with a better fit title . \n',
  '##does your apex dvd player only play dvd audio without video ? \n',
  '##or does it play audio and video but scrolling in black and white ? \n',
  '##before you try to return the player or waste hours calling apex tech support , or run the player over with your car , try these simple troubleshooting ideas first . \n',
  '##no picture : \n',
  '##hopefully you still have the remote control . \n',
  '##if you tossed it out the window , you need to fetch it . \n',
  '##using the remote control , press the i/p button located on the bottom right corner of the remote . \n',
  '##the i/p button switches the tv display between interlace and progressive . \n',
  '##if this doesnt bring back the picture , try pressing this button without playing a dvd . \n',
  '##if you dont get video back , now you can run the player over with your car ! \n',
  '##pict

In [ ]:
len(all_reviews)

638

From here, we try to input separate sentences to BERT and classify. (So do not use all_reviews)

In [ ]:
len(read)

10957

In [ ]:
# preprocessing
read_texts = []   #separate sentences
read_labels = []   #labels

# Delete titles
read_new = []
for i in range(len(read)):
    if read[i][:3] != '[t]':
        read_new.append(read[i])

for i in range(len(read_new)):
    start_index = read_new[i].index('##')
    #print(start_index)
    read_texts.append(read_new[i][start_index + 2:-2])
    if start_index == 0:
        read_labels.append('neu')
    else:
        senti = read_new[i][:start_index]
        #print(senti)
        pos = '+' in senti
        neg = '-' in senti
        #print(pos)
        if pos:
            read_labels.append('pos')
        elif neg:
            read_labels.append('neg')
        elif not pos and not neg:
            read_labels.append('neu')

In [ ]:
len(read_texts)

10319

In [ ]:
df = pd.DataFrame({'sentences': read_texts, 'labels': read_labels})
#pd.DataFrame({'name':s1, 'number':s2})

In [ ]:
df

,sentences,labels
0,"repost from january 13 , 2004 with a better fi...",neu
1,does your apex dvd player only play dvd audio ...,neu
2,or does it play audio and video but scrolling ...,neu
3,before you try to return the player or waste h...,neu
4,no picture :,neu
...,...,...
10314,"Finally I ran msconfig and went into the ""serv...",neu
10315,"This time, it started the uninstallation proce...",neg
10316,"I simply hate Symantec. I swear, if I could ha...",neg
10317,I have been a loyal Norton Anti-Virus and Inte...,neu


# 2. Multi class classification (sentiment)

Use BERT model to classify pos/neg. We want to avoid classify pos to neg and vice versa. We are quite open to neu.

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

In [ ]:
possible_labels = df.labels.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'neu': 0, 'pos': 1, 'neg': 2}

In [ ]:
df['labels'] = df.labels.replace(label_dict)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.labels.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.labels.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

#df.groupby(['sentences', 'labels', 'data_type']).count()

In [ ]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
#                                          do_lower_case=True)

#DistilBertTokenizerFast
#from transformers import RobertaTokenizer
#tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

#from transformers import BartTokenizer, BartModel
#tokenizer = BartTokenizer.from_pretrained('facebook/bart-large', do_lower_case=True)

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', 
                                          do_lower_case=True)

                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].sentences.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].sentences.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].labels.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].labels.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
#                                                      num_labels=len(label_dict),
#                                                      output_attentions=False,
#                                                      output_hidden_states=False)

model = BertForSequenceClassification.from_pretrained("bert-large-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.cuda()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class_strict(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print('Strict score:')
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)} or {len(y_preds[y_preds==label])/len(y_true)} \n')


def accuracy_per_class_relaxed(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for i in range(len(labels_flat)):
        if not labels_flat[i] + preds_flat[i] == 3:
             preds_flat[i] = labels_flat[i]

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print('Relaxed score:')
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)} or {len(y_preds[y_preds==label])/len(y_true)} \n')

In [ ]:
import random
import numpy as np

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/drive/MyDrive/GR/saved/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/549 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.7015452953581385
Validation loss: 0.5856319699705261
F1 Score (Weighted): 0.754973789664736


Epoch 2:   0%|          | 0/549 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.4762002802665985
Validation loss: 0.5795004879383697
F1 Score (Weighted): 0.7624015687037103


...


Epoch 3
Training loss: 0.312905904557047
Validation loss: 0.6496534461213141
F1 Score (Weighted): 0.7684292931098728


Epoch 4:   0%|          | 0/549 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.19937758392167493
Validation loss: 0.8316975360755453
F1 Score (Weighted): 0.7632868879502679


Epoch 5:   0%|          | 0/549 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.13606572292494046
Validation loss: 0.9278071910795784
F1 Score (Weighted): 0.7727205823660183


In [ ]:
# Compare all the models and pick one to continue
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for i in range(1,6):
    #model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
    #                                                      num_labels=len(label_dict),
    #                                                      output_attentions=False,
    #                                                      output_hidden_states=False)
    
    model = BertForSequenceClassification.from_pretrained("bert-large-uncased",
                                                          num_labels=len(label_dict),
                                                          output_attentions=False,
                                                          output_hidden_states=False)

    model.to(device)

    #model.load_state_dict(torch.load(f'/content/drive/MyDrive/GR/saved/finetuned_BERT_epoch_{i}.model', map_location=torch.device('cpu')))

    model.load_state_dict(torch.load('/content/drive/MyDrive/GR/saved/finetuned_BERT_large_uncased_0.983_0.966.model', map_location=torch.device('cpu')))
    print('epoch ', i)
    _, predictions, true_vals = evaluate(dataloader_validation)
    accuracy_per_class_relaxed(predictions, true_vals)
    #accuracy_per_class_strict(predictions, true_vals)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

epoch  1
Strict score:
Class: neu
Accuracy: 679/852 or 0.7969483568075117 

Strict score:
Class: pos
Accuracy: 356/458 or 0.777292576419214 

Strict score:
Class: neg
Accuracy: 155/238 or 0.6512605042016807 



Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

epoch  2
Strict score:
Class: neu
Accuracy: 679/852 or 0.7969483568075117 

Strict score:
Class: pos
Accuracy: 356/458 or 0.777292576419214 

Strict score:
Class: neg
Accuracy: 155/238 or 0.6512605042016807 



Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

epoch  3
Strict score:
Class: neu
Accuracy: 679/852 or 0.7969483568075117 

Strict score:
Class: pos
Accuracy: 356/458 or 0.777292576419214 

Strict score:
Class: neg
Accuracy: 155/238 or 0.6512605042016807 



Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

epoch  4


KeyboardInterrupt: ignored

Result: pos=0.983, neg=0.966 (relaxed)

# 3. Introduce raw data of Amazon reviews

Search for the most confident review sentences and save them as txt.

In this part, we first choose products with more than 100 reviews. Then for each product, choose the most confident 10 pos and neg sentences. Finally, save them in txt.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, SequentialSampler
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Settings

confidence_limit = 0.9  # Delete all the sentences whose confidence value is lower than this
review_num_lower_limit = 100  # Delete products whose review number is lower than or equal to this
review_num_upper_limit = 200  # Delete products whose review number is higher than or equal to this

Initialize BERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', 
                                          do_lower_case=True)

label_dict = {'neu': 0, 'pos': 1, 'neg': 2}
label_dict_inverse = {v: k for k, v in label_dict.items()}

# Use BERT model trained in the last step to classify sentences
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained("bert-large-uncased",
                                                       num_labels=len(label_dict),
                                                       output_attentions=False,
                                                       output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('/content/drive/MyDrive/GR/saved/finetuned_BERT_large_uncased_0.983_0.966.model', map_location=torch.device('cpu')))

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

<All keys matched successfully>

In [ ]:
def evaluate_raw(dataloader_raw):

    model.eval()
    predictions, true_vals = [], []
    
    for batch in dataloader_raw:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    predictions = np.concatenate(predictions, axis=0)
            
    return predictions

In [ ]:
def softmax(x, axis=1):
    row_max = x.max(axis=axis)
    row_max = row_max.reshape(-1, 1)
    hatx = x - row_max
    hatx_exp = np.exp(hatx)
    hatx_sum = np.sum(hatx_exp, axis=axis, keepdims=True)
    s = hatx_exp / hatx_sum
    return s

In [ ]:
review_path = '/content/drive/MyDrive/GR/Amazon raw data/MProductReviewsLatest.txt'
count = 0
products = []
ratings = []
reviews = []

with open(review_path) as f:
    try:
        for eachline in f:
            eachlist = eachline.split()
            products.append(eachlist[5])
            ratings.append(eachlist[7])
            reviews_not_joined = eachlist[10:]
            reviews.append(' '.join(reviews_not_joined))
            count += 1
    except:
        pass

In [ ]:
df_dict = {'products': products,
           'ratings': ratings,
           'reviews': reviews}
df_raw_reviews = pd.DataFrame(df_dict)

# Delete the titles
df_raw_reviews.drop(index=0, inplace=True)

df_raw_reviews

,products,ratings,reviews
1,B000FNS62Q,2,Disappointed Just receieved this and it doesn'...
2,B000FNS62Q,1,Canon camera cable I have two digital Canon Ca...
3,B000FNS62Q,5,need the special cable this cable looks like a...
4,B000FNS62Q,2,Nice looking...function not too good... it loo...
5,B000FJ7BM6,5,Hiarcs or Shredder? Or maybe even Rybka? Those...
...,...,...,...
320695,B00006LHM7,1,Caveat Emptor Based on my (and apparently othe...
320696,B00006LHM7,2,"planned obsolescence? Over the last year, we'v..."
320697,B00006LHM7,1,"Frustratingly bad experience Like others, I've..."
320698,B00006LHM7,5,"works well, nicely designed for home use. I pu..."


In [ ]:
# They contain most reviews, each with 2480, 2207, 1194, 1181 and 1089.
#most_sen_list = ['B000BKJZ9Q', 'B0002L5R78', 'B000A5TAT2', 'B0001IU5HY', 'B0007LJHVA']

#df_raw_reviews = df_raw_reviews[df_raw_reviews['products'] == 'B000CS7U62']
#df_raw_reviews

,products,ratings,reviews
6372,B000CS7U62,5,Greatest MP3 that i have had!! This creative i...
6373,B000CS7U62,5,I love my player I got my Zen about 6 months a...
6374,B000CS7U62,5,Zen Vision M - HAS IT ALL and I mean ALL This ...
6375,B000CS7U62,5,It truley is a vision to see and have This is ...
6376,B000CS7U62,5,"Versatile, great 30 GB storage My husband has ..."
...,...,...,...
6697,B000CS7U62,4,Truly an amazing gadget! I've had my Zen M for...
6698,B000CS7U62,4,A Truly Unbiased review! Very good portable me...
6699,B000CS7U62,5,iPod or Vision:M? Not black and white. Is the ...
6700,B000CS7U62,5,I now own 2 Vision:M players I had ordered one...


In [ ]:
# See how many reviews does a product contain
raw_statistic = df_raw_reviews.loc[:, 'products'].value_counts()
dict_raw = {'products': raw_statistic.index, 'counts': raw_statistic.values}
raw_statistic = pd.DataFrame(dict_raw)
raw_statistic

,products,counts
0,B000BKJZ9Q,2480
1,B0002L5R78,2207
2,B000A5TAT2,1194
3,B0001IU5HY,1181
4,B0007LJHVA,1089
...,...,...
18871,B0000XMXXO,1
18872,B0001HAFCA,1
18873,B0001HAFBQ,1
18874,B0000Y2VZS,1


In [ ]:
raw_statistic = raw_statistic[raw_statistic.counts >= review_num_lower_limit]
raw_statistic = raw_statistic[raw_statistic.counts <= review_num_upper_limit]

raw_statistic.reset_index(inplace=True)
del raw_statistic['index']
raw_statistic

,products,counts
0,B000094Q77,200
1,B0001EMLZ2,200
2,B00028Y826,200
3,B0009JMT38,200
4,B0001A967W,199
...,...,...
350,B0002U419Y,100
351,B000DT87W6,100
352,B0002229JQ,100
353,B000BKK7BG,100


In [ ]:
# This code is the integration of the following codes (blocks below). If setting upper limit to 200 and lower limit to 100, 
# it should make pos and neg for 355 products and take 5 hours to run on my Google Colab pro
for i in range(len(raw_statistic)):
    product_name = raw_statistic.products[i]
    df_one_product = df_raw_reviews[df_raw_reviews.products == product_name]

    # Explode the dataframe by segment the sentences
    df_one_product['reviews']=df_one_product['reviews'].map(lambda x:sent_tokenize(x.lower()))
    df_one_product=df_one_product.explode('reviews')

    # Make data
    encoded_data_raw = tokenizer.batch_encode_plus(
    df_one_product.reviews.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
    )

    input_ids_raw = encoded_data_raw['input_ids']
    attention_masks_raw = encoded_data_raw['attention_mask']
    dataset_raw = TensorDataset(input_ids_raw, attention_masks_raw)

    batch_size = 16
    dataloader_raw = DataLoader(dataset_raw, 
                                sampler=SequentialSampler(dataset_raw), 
                                batch_size=batch_size)
    
    # Get the labels
    predictions = evaluate_raw(dataloader_raw)
    preds_flat = np.argmax(predictions, axis=1).flatten()     # pos: 1, neg: 2, neu: 0(we do not care about neu)
    confidence = np.max(softmax(predictions), axis=1).flatten()

    sentiment_raw = []
    confidence_raw = []
    for j in range(len(preds_flat)):
        sentiment_raw.append(label_dict_inverse[preds_flat[j]])
        confidence_raw.append(confidence[j])

    # Add a column of sentiment classified by BERT model trained in the last part
    df_one_product.loc[:, 'sentiment'] = sentiment_raw
    df_one_product.loc[:, 'confidence'] = confidence_raw
    df_one_product = df_one_product[df_one_product['sentiment'].values != 'neu']
    df_one_product = df_one_product[df_one_product['confidence'].values >= confidence_limit]

    # Reset index
    df_one_product.reset_index(inplace=True)
    del df_one_product['index']

    df_one_product_pos = df_one_product[df_one_product['sentiment'] == 'pos']
    df_one_product_neg = df_one_product[df_one_product['sentiment'] == 'neg']
    df_one_product_pos = df_one_product_pos[df_one_product_pos['confidence'] >= 0.95]
    df_one_product_neg = df_one_product_neg[df_one_product_neg['confidence'] >= 0.95]

    # Concatenate sentences to str
    list_pos = [item for item in df_one_product_pos['reviews']]
    str_pos = ' '.join(list_pos)
    list_neg = [item for item in df_one_product_neg['reviews']]
    str_neg = ' '.join(list_neg)

    # Save the sentences to txt
    with open(f'/content/drive/MyDrive/GR/classified reviews/{product_name}_pos.txt', 'w') as f:
        f.write(str_pos)
        f.close()

    with open(f'/content/drive/MyDrive/GR/classified reviews/{product_name}_neg.txt', 'w') as f:
        f.write(str_neg)
        f.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batc

In [ ]:
# Here, we start with one product: B00008WEHZ, then iterate through all the df
product_name = 'B00008WEHZ'

df_one_product = df_raw_reviews[df_raw_reviews.products == product_name]
df_one_product

,products,ratings,reviews
287353,B00008WEHZ,1,Piece of junk It's hard to use and I am NOT im...
287354,B00008WEHZ,3,return i have returned this product
287355,B00008WEHZ,5,THE BEST!!! Hello everyone! This prodect is th...
287356,B00008WEHZ,1,"Display, what display? Backlight, contrast? Th..."
287357,B00008WEHZ,1,Sync feature is what matters and its not worki...
...,...,...,...
287448,B00008WEHZ,4,I'm happy now Except for the fact that the bot...
287449,B00008WEHZ,5,Excellent I have been using this PDA organizer...
287450,B00008WEHZ,4,Can't beat it for the price I was looking for ...
287451,B00008WEHZ,5,Simple Useful Personal Organizer I am a busy p...


In [ ]:
# Explode the dataframe by segment the sentences
df_one_product['reviews']=df_one_product['reviews'].map(lambda x:sent_tokenize(x.lower()))
df_one_product=df_one_product.explode('reviews')

df_one_product

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,products,ratings,reviews
287353,B00008WEHZ,1,piece of junk it's hard to use and i am not im...
287353,B00008WEHZ,1,it's going straight back...
287354,B00008WEHZ,3,return i have returned this product
287355,B00008WEHZ,5,the best!!!
287355,B00008WEHZ,5,hello everyone!
...,...,...,...
287451,B00008WEHZ,5,"the used, reconditioned version is even a bett..."
287452,B00008WEHZ,5,great product at a great price first and forem...
287452,B00008WEHZ,5,2. looking for e-mail synchronization.
287452,B00008WEHZ,5,3. looking for wireless connectivity or surfin...


In [ ]:
encoded_data_raw = tokenizer.batch_encode_plus(
    df_one_product.reviews.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_raw = encoded_data_raw['input_ids']
attention_masks_raw = encoded_data_raw['attention_mask']
dataset_raw = TensorDataset(input_ids_raw, attention_masks_raw)

batch_size = 16
dataloader_raw = DataLoader(dataset_raw, 
                            sampler=SequentialSampler(dataset_raw), 
                            batch_size=batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
predictions = evaluate_raw(dataloader_raw)
preds_flat = np.argmax(predictions, axis=1).flatten()     # pos: 1, neg: 2, neu: 0(we do not care about neu)
confidence = np.max(softmax(predictions), axis=1).flatten()

In [ ]:
confidence

array([0.86060995, 0.9860643 , 0.9365359 , 0.96958274, 0.98250157,
       0.97824997, 0.9332096 , 0.980973  , 0.98793924, 0.9837736 ,
       0.98575217, 0.9081096 , 0.6469504 , 0.9852041 , 0.6642286 ,
       0.9832613 , 0.9792062 , 0.94709146, 0.98541343, 0.86645705,
       0.71672326, 0.95996314, 0.98517215, 0.8295099 , 0.53943676,
       0.98243755, 0.6274996 , 0.92651105, 0.9489035 , 0.5504293 ,
       0.96293694, 0.97057915, 0.9235125 , 0.878495  , 0.9761588 ,
       0.9565748 , 0.8095904 , 0.8745401 , 0.855463  , 0.84406877,
       0.9555071 , 0.959359  , 0.93977904, 0.82940286, 0.9775284 ,
       0.96436733, 0.96468496, 0.96076024, 0.86060995, 0.9860643 ,
       0.9365359 , 0.96958274, 0.98250157, 0.97824997, 0.9332096 ,
       0.980973  , 0.98793924, 0.9837736 , 0.98575217, 0.9081096 ,
       0.6469504 , 0.9852041 , 0.6642286 , 0.9832613 , 0.9792062 ,
       0.94709146, 0.98541343, 0.86645705, 0.71672326, 0.95996314,
       0.98517215, 0.8295099 , 0.53943676, 0.98243755, 0.62749

In [ ]:
sentiment_raw = []
confidence_raw = []
for i in range(len(preds_flat)):
    sentiment_raw.append(label_dict_inverse[preds_flat[i]])
    confidence_raw.append(confidence[i])

In [ ]:
# Add a column of sentiment classified by BERT model trained in the last part
df_one_product.loc[:, 'sentiment'] = sentiment_raw
df_one_product.loc[:, 'confidence'] = confidence_raw
df_one_product = df_one_product[df_one_product['sentiment'].values != 'neu']
df_one_product = df_one_product[df_one_product['confidence'].values >= confidence_limit]

# Reset index
df_one_product.reset_index(inplace=True)
del df_one_product['index']

df_one_product

,products,ratings,reviews,sentiment,confidence
0,B00008WEHZ,5,this prodect is the best thing i have ever used!,pos,0.978250
1,B00008WEHZ,1,sync feature is what matters and its not worki...,neg,0.947091
2,B00008WEHZ,1,"actually it's not even connecting, am always g...",neg,0.959963
3,B00008WEHZ,1,"on the good side, other features are great!",pos,0.985172
4,B00008WEHZ,2,"i had this item for 1 week, dropped it one tim...",neg,0.926511
...,...,...,...,...,...
164,B00008WEHZ,4,"the read-out for the daily schedule is small, ...",neg,0.959601
165,B00008WEHZ,5,simple useful personal organizer i am a busy p...,pos,0.973232
166,B00008WEHZ,5,the enclosed softward makes it easy to synchro...,pos,0.932193
167,B00008WEHZ,5,the only drawback is that sometimes after sync...,neg,0.942753


In [ ]:
df_one_product_pos = df_one_product[df_one_product['sentiment'] == 'pos']
df_one_product_neg = df_one_product[df_one_product['sentiment'] == 'neg']

df_one_product_pos = df_one_product_pos[df_one_product_pos['confidence'] >= 0.95]
df_one_product_neg = df_one_product_neg[df_one_product_neg['confidence'] >= 0.95]

In [ ]:
df_one_product_neg

,products,ratings,reviews,sentiment,confidence
2,B00008WEHZ,1,"actually it's not even connecting, am always g...",neg,0.959963
5,B00008WEHZ,1,synchronization doesn't work this product prov...,neg,0.962937
8,B00008WEHZ,5,the one thing that bugs me a little is that th...,neg,0.959359
9,B00008WEHZ,5,only downfall was a small dark spot on the scr...,neg,0.964367
10,B00008WEHZ,1,synchro problems the software will not load co...,neg,0.964685
11,B00008WEHZ,1,there is no synchronization,neg,0.960760
14,B00008WEHZ,1,"actually it's not even connecting, am always g...",neg,0.959963
17,B00008WEHZ,1,synchronization doesn't work this product prov...,neg,0.962937
20,B00008WEHZ,5,the one thing that bugs me a little is that th...,neg,0.959359
21,B00008WEHZ,5,only downfall was a small dark spot on the scr...,neg,0.964367


In [ ]:
list_pos = [item for item in df_one_product_pos['reviews']]
str_pos = ' '.join(list_pos)

list_neg = [item for item in df_one_product_neg['reviews']]
str_neg = ' '.join(list_neg)

In [ ]:
with open('/content/drive/MyDrive/GR/classified reviews/B000CS7U62_pos.txt', 'w') as f:
    f.write(str_pos)
    f.close()

with open('/content/drive/MyDrive/GR/classified reviews/B000CS7U62_neg.txt', 'w') as f:
    f.write(str_neg)
    f.close()

In [ ]:
str_pos

'greatest mp3 that i have had!! otherwise this is an awesome player, all my friends go crazy over it it when they see it. it is a definite buy if your looking for something better than an ipod. also the music quality is awesome, i havent heard a single crack in th music, but for better quality you should get new earphones the ones the give are not that great sony has very good ones. i love my player i got my zen about 6 months ago and i have only had one problem with it. zen vision m - has it all and i mean all this little unit will not let you down. a powerful 30 gb can hold what you need...pay a little extra than you would for its "cousins," you will be very pleased! the sound quality is amazing. once you toy with the setup (initially it is a little difficult) and figure out all the bells and whistles you come to the conclusion that the zen vision m is a superstar among the players! it truley is a vision to see and have this is the first mp3 player i have ever owned. this player is v

In [ ]:
df_raw_reviews.loc[:, 'products'].value_counts()

B000BKJZ9Q    9520
B000CSWCQA    3560
B000ETVKHE    3479
B000BP50AY    3203
B000CS7U62    2494
              ... 
B000BWO79W       1
B000BP8AME       1
B000ER7O8U       1
B000C176S6       1
B000CPKEVI       1
Name: products, Length: 738, dtype: int64

In [ ]:
assert False

AssertionError: ignored

# 4. Summarization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import PegasusForConditionalGeneration, AutoTokenizer
import torch

# Tried different pegasus models and decide on google/pegasus-large
model_name = 'google/pegasus-large'
device = "cuda" if torch.cuda.is_available() else "cpu"
sum_tokenizer = AutoTokenizer.from_pretrained(model_name)
sum_model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

In [ ]:
with open('/content/drive/MyDrive/GR/classified reviews/B0000789TD_pos.txt') as f:
    for eachline in f:
        txt_pos = eachline

with open('/content/drive/MyDrive/GR/classified reviews/B0000789TD_neg.txt') as f:
    for eachline in f:
        txt_neg = eachline

In [ ]:
#txt_pos = 'PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow.'

In [ ]:
txt_neg

"but when i tried to put batteries in the remote the top screw would not unscrew. it seems as though the receiver on the mirror is defective. the only problem is...it won't attach to the seat in the car. my only problem is i don't think the lights are very bright (batteries are brand new). no way to turn it on without the remote which is bulky. (...) i'm not sure if we got a bad unit, but the remote control only works 1/3 of the time. the mirror also scratches pretty easy. my only complaint is that you have to position the remote just right to turn it on. but it doesn't install well. the musical selections are less than wonderful, and the arrangements are awful! however, consistent with other reviews, the remote control does not work half of the time and it eats up batteries. also, i have found that the remote will not work if it gets cold. sometimes the remote is left behind & i have no way to turn the mirror on. 1. the mirror scratches very easily, just try and breath on it and wipe.

In [ ]:
batch_pos = sum_tokenizer(txt_pos, truncation=True, padding="longest", return_tensors="pt").to(device)
translated_pos = sum_model.generate(**batch_pos)
sum_text_pos = sum_tokenizer.batch_decode(translated_pos, skip_special_tokens=True)

batch_neg = sum_tokenizer(txt_neg, truncation=True, padding="longest", return_tensors="pt").to(device)
translated_neg = sum_model.generate(**batch_neg)
sum_text_neg = sum_tokenizer.batch_decode(translated_neg, skip_special_tokens=True)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
sum_text_pos

["i love this mirror this is a great product... it fit perfectly in the middle of the bench seat in my trailblazer and the mirror is big enough that i can see my son perfectly...he absolutly loves this mirror, with the lights and music...i love the remote...so if he falls asleep i can turn it off/on without stopping the car!! i'm pretty pleased with battery usage, even with lights and sounds running. i'm pretty pleased with battery usage, even with lights and sounds running."]

In [ ]:
sum_text_neg

["(...) i'm not sure if we got a bad unit, but the remote control only works 1/3 of the time. not very durable or safe i bought the mirror when my son was 3 weeks old and he now 3 months old so needless to say it has only lasted 2 1/2 months before the strap that holds it in place snapped. not work in middle seat upon installing our car seat in the middle seat in the back, as directed, we found we absolutely could not get this mirror to stay with only using the clamp as it states in the directions."]

In [ ]:
# Save all the summarizations
import pandas as pd
import os
from pathlib import Path

In [ ]:
# Read all the text names in one folder
dir = '/content/drive/MyDrive/GR/classified reviews'
dataset_txt_paths = sorted([path for path in Path(dir).iterdir() if path.suffix in {'.txt'}])

# Check the number of files in the dataset
print('the dataset contains {} files'.format(len(dataset_txt_paths)))

the dataset contains 710 files


In [ ]:
# Check the names
for data_path in dataset_txt_paths:
    print(data_path)

/content/drive/MyDrive/GR/classified reviews/B00006LIQL_neg.txt
/content/drive/MyDrive/GR/classified reviews/B00006LIQL_pos.txt
/content/drive/MyDrive/GR/classified reviews/B00006LT11_neg.txt
/content/drive/MyDrive/GR/classified reviews/B00006LT11_pos.txt
/content/drive/MyDrive/GR/classified reviews/B00006SFM2_neg.txt
/content/drive/MyDrive/GR/classified reviews/B00006SFM2_pos.txt
/content/drive/MyDrive/GR/classified reviews/B00006W2QD_neg.txt
/content/drive/MyDrive/GR/classified reviews/B00006W2QD_pos.txt
/content/drive/MyDrive/GR/classified reviews/B00006Y0DA_neg.txt
/content/drive/MyDrive/GR/classified reviews/B00006Y0DA_pos.txt
/content/drive/MyDrive/GR/classified reviews/B0000789TD_neg.txt
/content/drive/MyDrive/GR/classified reviews/B0000789TD_pos.txt
/content/drive/MyDrive/GR/classified reviews/B00007AKDP_neg.txt
/content/drive/MyDrive/GR/classified reviews/B00007AKDP_pos.txt
/content/drive/MyDrive/GR/classified reviews/B00007E7RY_neg.txt
/content/drive/MyDrive/GR/classified rev

In [ ]:
for txt_path in dataset_txt_paths:
    ext = os.path.splitext(txt_path)[0][-14:] #The name of txt files
    #print(ext)
    with open(txt_path) as f:
        for eachline in f:
            before_sum = eachline
    with open('/content/drive/MyDrive/GR/summarizations/{}_sum.txt'.format(ext), 'w') as fin:
        batch = sum_tokenizer(before_sum, truncation=True, padding="longest", return_tensors="pt").to(device)
        translated = sum_model.generate(**batch)
        sum_text = sum_tokenizer.batch_decode(translated, skip_special_tokens=True)
        fin.write(sum_text[0])

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
sum_text[0]

"i didn't give it a full five stars largely because, on the whole, the quality of the puppetry (as opposed to others in the series like baby shakespeare and baby van gogh) is not of the same caliber and the toy sequences don't seem to have the same level of wit and humor that sparkles in the other videos."

In [ ]:
before_sum

'the headset paired with my desktop ok, it was recognized and seemed to install correctly. great buy i will be on the beaches of fla for 2 weeks in july and these headphones are gonna work out perfect. great price too! but in general they work well, and very importantly, they are comfortable (for me, anyway). everything went smoothly with the jabra bt620s setup, they are comfortable to wear, although i wish it had 2 more clicks up on the volume. great head set the head set works well with my samsung phone. i got a great price and the item arrived quickly. decent but goofy looking, work well with blackberry pearl pros: - easy pairing with blackberry pearl - good stereo sound, no crackling or interruptions - excellent for jogging, headphones are tight and band over the ears holds them in place very well when running, doing situps, etc. cons: - big and goofy looking, another reviewer said they make you look like princess leia which is very accurate, plus the nerdy blue lights that can be 

# 5. Evaluation

Reference: https://github.com/nlpaueb/SumQE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/nlpaueb/SumQE.git
!wget https://archive.org/download/sum-qe/BERT_DUC_all_Q1_Multi%20Task-1.h5
!wget https://archive.org/download/sum-qe/BERT_DUC_all_Q2_Multi%20Task-1.h5
!wget https://archive.org/download/sum-qe/BERT_DUC_all_Q3_Multi%20Task-1.h5
!wget https://archive.org/download/sum-qe/BERT_DUC_all_Q4_Multi%20Task-1.h5
!wget https://archive.org/download/sum-qe/BERT_DUC_all_Q5_Multi%20Task-1.h5

Cloning into 'SumQE'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 311 (delta 35), reused 25 (delta 25), pack-reused 266
Receiving objects: 100% (311/311), 4.25 MiB | 26.87 MiB/s, done.
Resolving deltas: 100% (170/170), done.
--2022-10-08 19:14:07--  https://archive.org/download/sum-qe/BERT_DUC_all_Q1_Multi%20Task-1.h5
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia801900.us.archive.org/20/items/sum-qe/BERT_DUC_all_Q1_Multi%20Task-1.h5 [following]
--2022-10-08 19:14:08--  https://ia801900.us.archive.org/20/items/sum-qe/BERT_DUC_all_Q1_Multi%20Task-1.h5
Resolving ia801900.us.archive.org (ia801900.us.archive.org)... 207.241.228.100
Connecting to ia801900.us.archive.org (ia801900.us.archive.org)|207.241.228.100|:443... connected.
HTTP req

In [ ]:
!pip uninstall -y tensorflow
!pip uninstall -y h5py
!pip install tensorflow==1.15
!pip install h5py==2.10.0
!pip install -r SumQE/requirements.txt
!pip install keras==2.3.1

Found existing installation: tensorflow 2.8.2+zzzcolab20220929150707
Uninstalling tensorflow-2.8.2+zzzcolab20220929150707:
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220929150707
Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Successfully uninstalled h5py-3.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 26 kB/s 
     |████████████████████████████████| 50 kB 7.7 MB/s 
     |████████████████████████████████| 3.8 MB 56.3 MB/s 
     |████████████████████████████████| 503 kB 71.2 MB/s 
     |████████████████████████████████| 4.1 MB 54.8 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=66d46a94bdee344d7c0609fd5111a12dd4402277d90d5b3a1b2a5899d840a750
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator

In [ ]:
# Save scores (Q1-Q5) for all the summarizations
import pandas as pd
import os
from pathlib import Path

In [ ]:
# Read all the text names in one folder
dir = '/content/drive/MyDrive/GR/summarizations'
dataset_txt_paths = sorted([path for path in Path(dir).iterdir() if path.suffix in {'.txt'}])

# Check the number of files in the dataset
print('the dataset contains {} files'.format(len(dataset_txt_paths)))

the dataset contains 710 files


In [ ]:
# Check the names
for data_path in dataset_txt_paths:
    print(data_path)

/content/drive/MyDrive/GR/summarizations/B00006LIQL_neg_sum.txt
/content/drive/MyDrive/GR/summarizations/B00006LIQL_pos_sum.txt
/content/drive/MyDrive/GR/summarizations/B00006LT11_neg_sum.txt
/content/drive/MyDrive/GR/summarizations/B00006LT11_pos_sum.txt
/content/drive/MyDrive/GR/summarizations/B00006SFM2_neg_sum.txt
/content/drive/MyDrive/GR/summarizations/B00006SFM2_pos_sum.txt
/content/drive/MyDrive/GR/summarizations/B00006W2QD_neg_sum.txt
/content/drive/MyDrive/GR/summarizations/B00006W2QD_pos_sum.txt
/content/drive/MyDrive/GR/summarizations/B00006Y0DA_neg_sum.txt
/content/drive/MyDrive/GR/summarizations/B00006Y0DA_pos_sum.txt
/content/drive/MyDrive/GR/summarizations/B0000789TD_neg_sum.txt
/content/drive/MyDrive/GR/summarizations/B0000789TD_pos_sum.txt
/content/drive/MyDrive/GR/summarizations/B00007AKDP_neg_sum.txt
/content/drive/MyDrive/GR/summarizations/B00007AKDP_pos_sum.txt
/content/drive/MyDrive/GR/summarizations/B00007E7RY_neg_sum.txt
/content/drive/MyDrive/GR/summarizations

In [ ]:
%cd /content/SumQE/

/content/SumQE


In [ ]:
import sys
sys.path.insert(0,'/content/SumQE')
%cd /content/SumQE/

import nltk
nltk.download('punkt')

# src/examples.py
import numpy as np

from keras.models import load_model
from nltk.tokenize import sent_tokenize

from src.BERT_experiments.BERT_model import BERT, custom_loss, set_quality_index
from src.vectorizer import BERTVectorizer

/content/SumQE


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Using TensorFlow backend.


In [ ]:
txt_names = []
scores = []
Qs = []
senti = []
for i in range(1,6):
    print('[INFO] Running for Q{}'.format(i))
    
    MODE = 'Multi Task-1'
    QUALITY = 'Q{}'.format(i)
    YEAR = 'all'

    model_path = '/content/BERT_DUC_{}_{}_{}.h5'.format(YEAR, QUALITY, MODE)

    # Set the quality index used in custom_loss
    set_quality_index(mode=MODE, quality=QUALITY)

    # Load the model
    model = load_model(model_path, custom_objects={'BERT': BERT, 'custom_loss': custom_loss})

    # Define the vectorizer
    vectorizer = BERTVectorizer()

    # For each model, do scoring
    for txt_path in dataset_txt_paths:
        ext = os.path.splitext(txt_path)[0][-18:-4] #The name of txt files
        txt_names.append(ext[:-4])
        Qs.append('Q{}'.format(i))
        senti.append(ext[-3:])
        #print(ext)
        with open(txt_path) as f:
            for eachline in f:
                after_sum = eachline

        # Summarization
        summary_token_ids = []
        for j, sentence in enumerate(sent_tokenize(after_sum)):
            sentence_tok = vectorizer.vectorize_inputs(sequence=sentence, i=j)
            summary_token_ids = summary_token_ids + sentence_tok
            
        # Transform the summary_tokens_ids into inputs --> (bpe_ids, mask, segments)
        inputs = vectorizer.transform_to_inputs(summary_token_ids)

        # Construct the dict that you will feed on your network. If you have multiple summaries,
        # you can update the lists and feed all of them together.
        test_dict = {
            'word_inputs': np.asarray([inputs[0, 0]]),
            'pos_inputs': np.asarray([inputs[1, 0]]),
            'seg_inputs': np.asarray([inputs[2, 0]])
        }
          
        output = model.predict(test_dict, batch_size=1)
        scores.append(round(output[0][0], 4))


scores_df = pd.DataFrame({'product_name':txt_names,'QE score':scores,'measurement':Qs, 'sentiment':senti})
scores_df.to_csv('/content/drive/MyDrive/GR/scores.csv', sep = ',', index = True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "




/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting spar

In [ ]:
scores_df

,product_name,QE score,measurement,sentiment
0,B00006LIQL,0.4971,Q1,neg
1,B00006LIQL,0.2175,Q1,pos
2,B00006LT11,0.2833,Q1,neg
3,B00006LT11,0.4264,Q1,pos
4,B00006SFM2,0.7846,Q1,neg
...,...,...,...,...
3545,B000EVOOO8,0.3117,Q5,pos
3546,B000EW8074,0.0713,Q5,neg
3547,B000EW8074,0.3520,Q5,pos
3548,B000FJ20CM,0.3429,Q5,neg


In [ ]:
assert False

# Another amazon based sentiment analysis

In [ ]:
assert False

In [ ]:
assert False

In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np

In [ ]:
num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", num_gpus_available)

In [ ]:
possible_labels = df.labels.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

In [ ]:
df['labels'] = df.labels.replace(label_dict)

In [ ]:
df

In [ ]:
reviews = df['sentences'].values.tolist()
labels = df['labels'].tolist()

In [ ]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=0.2)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# use an example to see if the tokenizer works
tokenizer([training_sentences[0]], truncation=True,
           padding=True, max_length=128)

In [ ]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)
train_dataset = tf.data.Dataset.from_tensor_slices((
                            dict(train_encodings),
                            training_labels
                            ))
val_dataset = tf.data.Dataset.from_tensor_slices((
                            dict(val_encodings),
                            validation_labels
                            ))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=3)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=5,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

In [ ]:
model.save_pretrained("/content/drive/MyDrive/GR")
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/GR")

In [ ]:
assert False

Read the Amazon raw data

Insert BERT classifier here

In [ ]:
!pip install transformers

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import csv

from tabulate import tabulate
from tqdm import trange
import random

In [ ]:
text = df.sentences.values
labels = df.labels.values

In [ ]:
# bert-base-uncased
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

from transformers import AutoTokenizer, AutoModelForSequenceClassification
#tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract')
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)

In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 128,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids']) 
    attention_masks.append(encoding_dict['attention_mask'])

print(token_id)

token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

In [ ]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [ ]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
    '''
    Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
    '''
    preds = np.argmax(preds, axis = 1).flatten()
    labels = labels.flatten()
    tp = b_tp(preds, labels)
    tn = b_tn(preds, labels)
    fp = b_fp(preds, labels)
    fn = b_fn(preds, labels)
    b_accuracy = (tp + tn) / len(labels)
    # The if else usage is also a grammar sugar for python
    b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
    b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
    b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
    
    return b_accuracy, b_precision, b_recall, b_specificity

In [ ]:
# Load the BertForSequenceClassification model, normally, we use BERTmodel (TFBERTmodel, BertTokenizer)
#model = BertForSequenceClassification.from_pretrained(
#    'alvaroalon2/biobert_diseases_ner',
#    num_labels = 2,
#    output_attentions = False,
#    output_hidden_states = False,
#)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

# trange is short for tqdm(range(...)), 
for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('- Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('- Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '- Validation Precision: NaN')
    print('- Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '- Validation Recall: NaN')
    print('- Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '- Validation Specificity: NaN')

In [ ]:
new_sentence = 'This device is wonderful, I cannot help appraise it'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Yes' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'No'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

End

In [ ]:
assert False

Tensorflow

In [ ]:
!pip install tensorflow-text

In [ ]:
import pandas as pd
import numpy as np
import csv

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
# This is a quite standard use of train_test_split, input x and y. Note stratify avoids the imbalance in the labels. (Our labels itself is imbalanced)
X_train, X_test, y_train, y_test = train_test_split(df['sentences'],df['labels'], stratify=df['labels'], test_size = 0.1)
X_train.head(4)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
#bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3')

bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')
#bert_encoder = hub.KerasLayer('https://tfhub.dev/google/experts/bert/pubmed/2')
#bert_encoder = hub.KerasLayer('https://tfhub.dev/jeongukjae/roberta_en_cased_L-24_H-1024_A-16/1')
#bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_cased_L-24_H-1024_A-16/4')
#bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/4')
#bert_encoder = hub.KerasLayer('https://tfhub.dev/google/albert_base/3')

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
#l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(50, activation='relu', name="layer_mid")(outputs['pooled_output'])
l = tf.keras.layers.Dense(3, activation='softmax', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=2, batch_size = 16, validation_split = 0.11)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
print(y_predicted)

In [ ]:
y_predicted = list(y_predicted)

for i in range(len(y_predicted)):
    if y_predicted[i] >= 0.3333:
        y_predicted[i] = 1
    elif y_predicted[i] <= -0.3333:
        y_predicted[i] = -1
    else:
        y_predicted[i] = 0
#y_predicted = [1 if y_predicted[i] >= 0.5 else 0 for i in range(len(y_predicted))]

In [ ]:
real = y_test.to_list()

In [ ]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
    '''
    Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
    '''
    #preds = np.argmax(preds, axis = 1).flatten()
    #labels = labels.flatten()
    tp = b_tp(preds, labels)
    tn = b_tn(preds, labels)
    fp = b_fp(preds, labels)
    fn = b_fn(preds, labels)
    b_accuracy = (tp + tn) / len(labels)
    # The if else usage is also a grammar sugar for python
    b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
    b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
    b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
    
    return b_accuracy, b_precision, b_recall, b_specificity

In [ ]:
val_accuracy = []
val_precision = []
val_recall = []
val_specificity = []

b_accuracy, b_precision, b_recall, b_specificity = b_metrics(y_predicted, real)
val_accuracy.append(b_accuracy)
# Update precision only when (tp + fp) !=0; ignore nan
if b_precision != 'nan': val_precision.append(b_precision)
# Update recall only when (tp + fn) !=0; ignore nan
if b_recall != 'nan': val_recall.append(b_recall)
# Update specificity only when (tn + fp) !=0; ignore nan
if b_specificity != 'nan': val_specificity.append(b_specificity)

#print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
print('- Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
print('- Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '- Validation Precision: NaN')
print('- Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '- Validation Recall: NaN')
print('- Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '- Validation Specificity: NaN')

End

In [ ]:
assert False

In [ ]:
def remove_extra_spaces(text):
    return ' '.join(text.split())

In [ ]:
def clean_data(data):
    data_sen = []
    labels = []
    for i in range(len(data)):
        if data[i].count('##') == 0:
            continue
        if data[i].count('##') == 2:
            print(data[i])
            print(i)
            continue
        assert data[i].count('##') == 1, "This sentence contains {} '##'".format(data[i].count('##'))
        index_start = data[i].index('##')
        #data[i] = remove_extra_spaces(data[i])
        data_sen.append(data[i][index_start + 3: -1])
        if data[i][:index_start] == '':
            labels.append(data[i][:index_start])
        else:
            labels.append(data[i][:index_start - 1])
    return data_sen, labels

In [ ]:
def make_detailed_label(labels):
    label_a = []
    label_v = []
    label_emotion = []
    topic_total = []

    for i in range(len(labels)):
        splitted = labels[i].split()
        for j in range(len(splitted)):
            if '[' not in splitted[j]:
                topic_total.append(splitted[j])
            else:
                index = splitted[j].index('[')
                topic_total.append(splitted[j][:index])
    print(list(set(topic_total)))
    print(len(list(set(topic_total))))
    print(len(topic_total))
        #if '[a]' in labels[i]:
        #    label_a.append(1)
        #else:
        #    label_a.append(0)
        #if '[v]' in labels[i]:
        #    label_v.append(1)
        #else:
        #    label_v.append(0)
        #print(splitted)
        

In [ ]:
data_sen, labels = clean_data(read)
make_detailed_label(labels)

In [ ]:
df

In [ ]:
labels

In [ ]:
read